In [1]:
sc

In [137]:
import datetime
from pyspark.sql.functions import *
from pyspark.sql.functions import udf
from pyspark.sql.types import BooleanType
from geopy.distance import vincenty
from pyspark.sql import Row
import csv
from pyspark.sql import SQLContext
import shapely as shp
from shapely.geometry import Point, Polygon
import shapely as shp
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
import geopandas as gpd
import seaborn as sns


In [74]:
airports = sqlContext.read.load(
    'Data/Airport Codes8.csv', format='csv', header=True, inferSchema=True)

In [75]:
for num, i in enumerate(airports.dtypes):
    print i, num

('Airport (IATA)', 'string') 0
('Airport (Name)', 'string') 1
('Airport (City)', 'string') 2
('Country (Name)', 'string') 3
('Latitude', 'double') 4
('Longitude', 'double') 5


In [76]:
airports = airports.select('*').withColumnRenamed('Airport (IATA)', 'Airport')
airports = airports.select('*').withColumnRenamed('Airport (Name)', 'Name')
airports = airports.select('*').withColumnRenamed('Airport (City)', 'City')
airports = airports.drop('Country (Name)')

In [77]:
airports.show(5)

+-------+--------------------+-------------+----------+-----------+
|Airport|                Name|         City|  Latitude|  Longitude|
+-------+--------------------+-------------+----------+-----------+
|    04G|   Lansdowne Airport|   Youngstown|41.1304722|-80.6195833|
|    06A|Moton Field Munic...|     Tuskegee|32.4605722|-85.6800278|
|    06C| Schaumburg Regional|   Schaumburg|41.9893408|-88.1012428|
|    06N|     Randall Airport|   Middletown| 41.431912|-74.3915611|
|    09J|Jekyll Island Air...|Jekyll Island|31.0744722|-81.4277778|
+-------+--------------------+-------------+----------+-----------+
only showing top 5 rows



In [78]:
busyairports = sqlContext.read.load(
    'Output/MostBussyAirport.csv', format='csv', header=True, inferSchema=True)
for num, i in enumerate(busyairports.dtypes):
    print i, num

('_c0', 'int') 0
('Airport', 'string') 1
('origin_count', 'double') 2
('dest_count', 'double') 3
('Total_Flights', 'double') 4


In [79]:
busyairports.show(10)

+---+-------+------------+----------+-------------+
|_c0|Airport|origin_count|dest_count|Total_Flights|
+---+-------+------------+----------+-------------+
|  0|    ATL|   1133578.0| 1133592.0|    2267170.0|
|  1|    ORD|    844654.0|  844780.0|    1689434.0|
|  2|    DFW|    734953.0|  734956.0|    1469909.0|
|  3|    DEN|    664532.0|  664427.0|    1328959.0|
|  4|    LAX|    647316.0|  647348.0|    1294664.0|
|  5|    SFO|    501429.0|  501417.0|    1002846.0|
|  6|    PHX|    479337.0|  479374.0|     958711.0|
|  7|    IAH|    471135.0|  471237.0|     942372.0|
|  8|    LAS|    433610.0|  433648.0|     867258.0|
|  9|    SEA|    362626.0|  362641.0|     725267.0|
+---+-------+------------+----------+-------------+
only showing top 10 rows



In [80]:
busyairports = busyairports.select('*', coalesce(busyairports["origin_count"], lit(0.0)).alias('clean_origin_count'))
busyairports = busyairports.drop('origin_count')
busyairports = busyairports.select('*', coalesce(busyairports["dest_count"], lit(0.0)).alias('clean_dest_count'))
busyairports = busyairports.drop('dest_count')
busyairports = busyairports.select('*', coalesce(busyairports["Total_Flights"], lit(0.0)).alias('clean_Total_Flights'))
busyairports = busyairports.drop('Total_Flights')
busyairports = busyairports.select('*', (busyairports.clean_origin_count+busyairports.clean_dest_count).alias('Total_Flights') )
busyairports.show()

+---+-------+------------------+----------------+-------------------+-------------+
|_c0|Airport|clean_origin_count|clean_dest_count|clean_Total_Flights|Total_Flights|
+---+-------+------------------+----------------+-------------------+-------------+
|  0|    ATL|         1133578.0|       1133592.0|          2267170.0|    2267170.0|
|  1|    ORD|          844654.0|        844780.0|          1689434.0|    1689434.0|
|  2|    DFW|          734953.0|        734956.0|          1469909.0|    1469909.0|
|  3|    DEN|          664532.0|        664427.0|          1328959.0|    1328959.0|
|  4|    LAX|          647316.0|        647348.0|          1294664.0|    1294664.0|
|  5|    SFO|          501429.0|        501417.0|          1002846.0|    1002846.0|
|  6|    PHX|          479337.0|        479374.0|           958711.0|     958711.0|
|  7|    IAH|          471135.0|        471237.0|           942372.0|     942372.0|
|  8|    LAS|          433610.0|        433648.0|           867258.0|     86

In [81]:
busyairports.count()

336

In [82]:
busyairports.filter(busyairports.clean_Total_Flights==0).show()

+---+-------+------------------+----------------+-------------------+-------------+
|_c0|Airport|clean_origin_count|clean_dest_count|clean_Total_Flights|Total_Flights|
+---+-------+------------------+----------------+-------------------+-------------+
|330|    ENV|               1.0|             0.0|                0.0|          1.0|
|331|   DEST|               0.0|            35.0|                0.0|         35.0|
|332| ORIGIN|              35.0|             0.0|                0.0|         35.0|
|333|    MCN|               1.0|             0.0|                0.0|          1.0|
|334|    EFD|               1.0|             0.0|                0.0|          1.0|
|335|    BFF|               1.0|             0.0|                0.0|          1.0|
+---+-------+------------------+----------------+-------------------+-------------+



In [83]:
busyairports.filter(busyairports.Total_Flights==0).show()

+---+-------+------------------+----------------+-------------------+-------------+
|_c0|Airport|clean_origin_count|clean_dest_count|clean_Total_Flights|Total_Flights|
+---+-------+------------------+----------------+-------------------+-------------+
+---+-------+------------------+----------------+-------------------+-------------+



In [84]:
busyairports = busyairports.filter(busyairports.Airport!='DEST')
busyairports = busyairports.filter(busyairports.Airport!='ORIGIN')
busyairports.count()

334

In [85]:
busyairports = busyairports.drop('clean_Total_Flights')
busyairports = busyairports.drop('_c0')
busyairports = busyairports.select('*').withColumnRenamed('clean_dest_count', 'dest_count')
busyairports = busyairports.select('*').withColumnRenamed('clean_origin_count', 'origin_count')
busyairports.show()

+-------+------------+----------+-------------+
|Airport|origin_count|dest_count|Total_Flights|
+-------+------------+----------+-------------+
|    ATL|   1133578.0| 1133592.0|    2267170.0|
|    ORD|    844654.0|  844780.0|    1689434.0|
|    DFW|    734953.0|  734956.0|    1469909.0|
|    DEN|    664532.0|  664427.0|    1328959.0|
|    LAX|    647316.0|  647348.0|    1294664.0|
|    SFO|    501429.0|  501417.0|    1002846.0|
|    PHX|    479337.0|  479374.0|     958711.0|
|    IAH|    471135.0|  471237.0|     942372.0|
|    LAS|    433610.0|  433648.0|     867258.0|
|    SEA|    362626.0|  362641.0|     725267.0|
|    MSP|    358153.0|  358101.0|     716254.0|
|    MCO|    356161.0|  356166.0|     712327.0|
|    BOS|    349907.0|  349895.0|     699802.0|
|    DTW|    348364.0|  348422.0|     696786.0|
|    EWR|    337533.0|  337652.0|     675185.0|
|    CLT|    332815.0|  332838.0|     665653.0|
|    SLC|    323378.0|  323397.0|     646775.0|
|    LGA|    311730.0|  311689.0|     62

In [86]:
locAirports = airports.join(busyairports, on = "Airport", how = 'inner')
locAirports.show()

+-------+--------------------+-------------+----------+------------+------------+----------+-------------+
|Airport|                Name|         City|  Latitude|   Longitude|origin_count|dest_count|Total_Flights|
+-------+--------------------+-------------+----------+------------+------------+----------+-------------+
|    ABE|  Lehigh Valley Intl|    Allentown| 40.652083|  -75.440806|      6520.0|    6524.0|      13044.0|
|    ABI|        Abilene Rgnl|      Abilene| 32.411319|  -99.681897|      5523.0|    5522.0|      11045.0|
|    ABQ|Albuquerque Inter...|  Albuquerque|35.0402222|-106.6091944|     64610.0|   64602.0|     129212.0|
|    ABR|Aberdeen Regional...|     Aberdeen|   45.4491|    -98.4218|      2237.0|    2236.0|       4473.0|
|    ABY|Southwest Georgia...|       Albany|   31.5355|    -84.1945|      2968.0|    2968.0|       5936.0|
|    ACK|       Nantucket Mem|    Nantucket| 41.253053|  -70.060181|      1529.0|    1529.0|       3058.0|
|    ACT|           Waco Rgnl|       

In [87]:
locAirports.count()

334

In [55]:
for num, i in enumerate(locAirports.dtypes):
    print i, num

('Airport', 'string') 0
('Airport (Name)', 'string') 1
('Airport (City)', 'string') 2
('Country (Name)', 'string') 3
('Latitude', 'double') 4
('Longitude', 'double') 5
('_c0', 'int') 6
('clean_origin_count', 'double') 7
('clean_dest_count', 'double') 8
('Total_Flights', 'double') 9


In [133]:
locAirports.toPandas().to_csv('Output/Airports_Location_Counts.csv')

In [140]:
pdlocAirports = locAirports.toPandas()

In [141]:
geometry = []
for air in pdlocAirports.Airport:
    geometry.append(shp.geometry.Point(pdlocAirports.loc[pdlocAirports.Airport==air,'Longitude'],pdlocAirports.loc[pdlocAirports.Airport==air,'Latitude']))

In [142]:
crs = {'init': 'epsg:3857'}
pdlocAirports = gpd.GeoDataFrame(pdlocAirports,crs=crs, geometry=geometry)

In [144]:
pdlocAirports.head(3)

,Airport,Name,City,Latitude,Longitude,origin_count,dest_count,Total_Flights,geometry
0,ABE,Lehigh Valley Intl,Allentown,40.652083,-75.440806,6520.0,6524.0,13044.0,POINT (-75.440806 40.652083)
1,ABI,Abilene Rgnl,Abilene,32.411319,-99.681897,5523.0,5522.0,11045.0,POINT (-99.68189700000001 32.411319)
2,ABQ,Albuquerque International Sunport,Albuquerque,35.040222,-106.609194,64610.0,64602.0,129212.0,POINT (-106.6091944 35.0402222)


In [148]:
with open('Data/Airport_Codes_Lat_Lon.geojson', 'w') as f:
    f.write(pdlocAirports.to_json())